#Install Required Libraries:
This cell installs Keras Tuner, a library for hyperparameter optimization of Keras models.

In [5]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 1.5 MB/s eta 0:00:00


#Import Libraries:
This cell imports the necessary libraries, including Keras for building neural networks, Keras Tuner for hyperparameter tuning, and scikit-learn for data preprocessing and evaluation metrics.

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, precision_score, recall_score
from sklearn.datasets import load_digits
from keras.models import Sequential
from keras.layers import Dense
from keras.regularizers import l2
from kerastuner.tuners import RandomSearch

<ipython-input-6-25376277e413>:10: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


#Load and Preprocess the MNIST Dataset:
The MNIST dataset is loaded from sklearn.datasets. The data is split into training and test sets, and the features are standardized using StandardScaler.

In [7]:
# Load the MNIST dataset
mnist = load_digits()


# Pandas data frame with feature vectors
X = mnist.data

# Labels
y = mnist.target

# Labels converted to integers
y = y.astype(int)
print("Feature vectors shape:", X.shape)
print("Labels shape:", y.shape)

Feature vectors shape: (1797, 64)
Labels shape: (1797,)


In [8]:
# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the data for SVC and Logistic Regression
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#Define the Model Building Function:
This function defines a model-building function compatible with Keras Tuner. It includes an adjustable number of units in the first Dense layer and an L2 regularization parameter. The output layer uses a softmax activation function to classify the digits.

In [11]:
# Define the model
def build_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu', input_shape=(64,), kernel_regularizer=l2(hp.Float('l2', min_value=0.001, max_value=0.1, step=0.01))))
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

#Hyperparameter Tuning Using Random Search:
Keras Tuner's RandomSearch is used to search for the best hyperparameters. The tuning is performed over 10 trials, with 3 executions per trial, optimizing for validation accuracy.

In [12]:
# Hyperparameter tuning
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=3,
    directory='my_dir',
    project_name='helloworld')

tuner.search(X_train_scaled, y_train, epochs=10, validation_data=(X_test_scaled, y_test))


Trial 10 Complete [00h 00m 27s]
val_accuracy: 0.9675925771395365

Best val_accuracy So Far: 0.9814814726511637
Total elapsed time: 00h 04m 13s


#Retrieve and Evaluate the Best Model:
The best model identified by Keras Tuner is retrieved and evaluated on the test set.

In [13]:
# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


# Classification Report and Confusion Matrix:
The predictions on the test set are used to generate a classification report and confusion matrix, providing detailed metrics for the model's performance.

In [14]:
# Evaluate the best model
loss, accuracy = best_model.evaluate(X_test_scaled, y_test)
print('Test accuracy:', accuracy)

# Print classification report and confusion matrix
y_pred = best_model.predict(X_test_scaled)
y_pred_class = np.argmax(y_pred, axis=1)
print(classification_report(y_test, y_pred_class))
print(confusion_matrix(y_test, y_pred_class))

12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step - accuracy: 0.9870 - loss: 0.1740 
Test accuracy: 0.9833333492279053
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
              precision    recall  f1-score   support

           0       0.97      1.00      0.99        33
           1       1.00      1.00      1.00        28
           2       1.00      1.00      1.00        33
           3       1.00      0.97      0.99        34
           4       1.00      1.00      1.00        46
           5       0.98      0.94      0.96        47
           6       0.97      0.97      0.97        35
           7       1.00      0.97      0.99        34
           8       1.00      1.00      1.00        30
           9       0.93      1.00      0.96        40

    accuracy                           0.98       360
   macro avg       0.99      0.98      0.98       360
weighted avg       0.98      0.98      0.98       360

[[33  0  0  0  0  0  0  0  0  0]
 [ 0 28  0  0  0  0  0  0  0  0]
 [ 0  0 33  0  0  0  0  0 